`Runtime > Change runtime type > TPU v2`

In [1]:
!pip install numerapi
!pip install numerai-tools
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 880.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.1.3 which is incompatible.
CPU times: user 99.8 ms, sys: 32.8 ms, total: 133 ms
Wall time: 16.8 s


In [1]:
from numerapi import NumerAPI
from numerai_tools.scoring import numerai_corr, correlation_contribution

import pandas as pd
import json

import matplotlib.pyplot as plt

api = NumerAPI()

## Download v5.0 data

In [4]:
VERSION = 5.0
datasets = [dataset for dataset in api.list_datasets() if (dataset.startswith(f'v{VERSION}')) & ~("example" in dataset)]
datasets

CPU times: user 9.9 ms, sys: 0 ns, total: 9.9 ms
Wall time: 310 ms


['v5.0/features.json',
 'v5.0/train.parquet',
 'v5.0/train_benchmark_models.parquet',
 'v5.0/validation.parquet',
 'v5.0/validation_benchmark_models.parquet']

In [5]:
%%time
for dataset in datasets:
  api.download_dataset(dataset)

v5.0/features.json: 480kB [00:00, 2.21MB/s]                           
v5.0/train.parquet: 2.37GB [01:53, 20.8MB/s]                            
v5.0/train_benchmark_models.parquet: 81.7MB [00:04, 19.2MB/s]                            
v5.0/validation.parquet: 3.18GB [02:57, 18.0MB/s]                            
v5.0/validation_benchmark_models.parquet: 133MB [00:06, 22.1MB/s]                           

CPU times: user 50.6 s, sys: 10.5 s, total: 1min 1s
Wall time: 5min 3s


## Read data

In [10]:
%%time
train = pd.read_parquet("v5.0/train.parquet")
train.era = train.era.astype(int)
train.drop("data_type", axis=1, inplace=True)

validation = pd.read_parquet("v5.0/validation.parquet")
validation.era = validation.era.astype(int)
validation = validation.loc[validation["data_type"]=="validation"]
validation.drop("data_type", axis=1, inplace=True)

CPU times: user 2min 49s, sys: 2min 21s, total: 5min 11s
Wall time: 26.6 s


In [1]:
train

NameError: name 'train' is not defined

In [9]:
validation

,era,data_type,feature_shaded_hallucinatory_dactylology,feature_itinerant_hexahedral_photoengraver,feature_prudent_pileate_oven,feature_subalpine_apothegmatical_ajax,feature_pistachio_atypical_malison,feature_symmetrical_spongy_tricentenary,feature_ungrounded_transpontine_winder,feature_aseptic_eely_hemiplegia,...,target_teager2b_20,target_teager2b_60,target_tyler_20,target_tyler_60,target_victor_20,target_victor_60,target_waldo_20,target_waldo_60,target_xerxes_20,target_xerxes_60
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,575,validation,2,0,4,0,3,0,4,1,...,0.75,0.50,0.75,0.50,0.50,0.50,0.50,0.50,0.75,0.50
n001e1318d5072ac,575,validation,1,4,2,2,1,3,3,0,...,0.00,0.25,0.25,0.25,0.00,0.25,0.25,0.25,0.25,0.25
n002a9c5ab785cbb,575,validation,1,2,2,3,1,1,3,0,...,0.50,0.75,0.50,0.50,0.50,0.75,0.50,0.75,0.50,0.50
n002ccf6d0e8c5ad,575,validation,2,4,2,4,2,4,3,2,...,0.00,0.00,0.00,0.00,0.25,0.25,0.25,0.25,0.00,0.00
n0041544c345c91d,575,validation,0,3,2,0,4,1,2,2,...,0.75,0.50,0.75,0.75,0.75,0.50,0.50,0.50,0.50,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffd1d288395156b,1125,validation,1,1,3,2,2,3,4,4,...,0.50,NaN,0.50,NaN,0.25,NaN,0.25,NaN,0.25,NaN
nffdd88f9a9554be,1125,validation,0,0,3,0,2,1,0,1,...,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN
nffe07ce4a95ca46,1125,validation,1,1,0,3,2,4,3,0,...,0.75,NaN,0.75,NaN,1.00,NaN,0.75,NaN,0.75,NaN
